In [ ]:
import pandas as pd
import plotly.express as px

from risk_data import get_factor_data
from risk_chart import px_format
from risk_event_study import draw_event_study


In [ ]:
def get_event_window(
    df: pd.Series | pd.DataFrame,
    event_date: str | pd.Timestamp,
    before: int = 21,
    after: int = 63
) -> pd.Series | pd.DataFrame:
    """
    Select a window of rows from a date-indexed DataFrame around an event date.

    Parameters:
    ----------
    df : pd.DataFrame
        DataFrame with a DateTimeIndex, assumed sorted.
    event_date : str | pd.Timestamp
        The target event date.
    before : int
        Number of rows before the event date to include.
    after : int
        Number of rows after the event date to include.

    Returns:
    -------
    pd.DataFrame
        Sliced DataFrame with [before rows] before and [after rows] after the next available event date.
    """
    
    event_date = pd.Timestamp(event_date)
    # Find next available index position (bfill means ≥ event_date)
    
    event_idx = df.index.get_indexer([event_date], method='bfill')[0]
    if event_idx == -1:
        raise ValueError(f"No date on or after {event_date} found in index!")
    
    start_idx = max(event_idx - before, 0)
    end_idx = event_idx + after + 1  # +1 because iloc is exclusive on end
    return df.iloc[start_idx:end_idx]


def run_event_study(returns_df: pd.Series | pd.DataFrame, 
                    event_list: list[tuple[str, pd.Timestamp]], 
                    before: int = 21, 
                    after:  int = 63) -> tuple:
    """
    Perform event study using specific factor names tied to events.

    Parameters:
    - returns_df: pd.DataFrame (date index, return columns)
    - event_list: list of (factor_name, event_date)
    - before: days before event
    - after: days after event

    Returns:

    """
    # offsets = range(-before, after + 1)
    # event_windows = []

    _list = []
    for factor_name, event_date in event_list:
        _list.append(
            get_event_window(returns_df[factor_name],
                             event_date, 
                             before=before, 
                             after=after)
            .rename('returns')
            .to_frame()
            # .assign(day_offset=range(-before, after + 1)) # THIS MIGHT BE TOO LONG
            .assign(day_offset=lambda df_: range(-before, -before + len(df_)),
                    factor_name=factor_name,
                    event_date=event_date,
                    cret = lambda df: df.groupby(['factor_name', 'event_date'])['returns'].cumsum(),
                    )
            .reset_index()
            .set_index('day_offset')
        )
    return pd.concat(_list)


In [ ]:
# event_study = run_event_study(ret, event_list, before=before, after=after)
# event_study
# px.line(event_study.reset_index(), x='day_offset', y='cret', color='event_date', template='plotly_white')

In [ ]:
factor_data = get_factor_data() #read_cache=False)
ret = factor_data.ret.to_pandas()

In [ ]:
df = factor_data.ret.sel(factor_name=['^VIX'])
df = df.to_pandas().div(10_000).add(1).cumprod().sub(1)
px.line(df)

In [ ]:
df = factor_data.ret.sel(factor_name=['TRADEWAR'])
px.line(df)

In [ ]:

# def flatten_multiindex(index, sep='_'):
#     return index.map(lambda x: sep.join(map(str, x)))

# def draw_event_study(ret, event_list, before, after):
#     # TODO: Run cumulative return inside the event study
#     event_study = run_event_study(ret, event_list, before=before, after=after)
#     # event_study.reset_index().pivot(index='day_offset', columns=('factor_name', 'event_date'), values='returns')
#     df_cum = event_study.reset_index().pivot(index='day_offset', columns=['factor_name', 'event_date'], values='returns').cumsum()
#     df_cum.columns = flatten_multiindex(df_cum.columns, sep=';')

#     # event_study -= event_study.loc[0]
#     df_cum -= df_cum.loc[0]
#     fig = px.line(df_cum/100, template='plotly_white', title='Event Study')
#     return px_format(fig)

    

event_list_hispy = [('SPY', '2018-01-26'),  # Align SPY peak
                   ('SPY', '2025-02-19'),]
event_list_jan1 = [('^VIX', '2018-01-01'),  # Start Jan 1
                   ('^VIX', '2025-01-01'),]
event_list_hivix = [('^VIX', '2018-02-04'), # Align VIX peak
                    ('^VIX', '2025-04-08')]
event_list_hivix2 = [('^VIX', '2018-01-31'), # Align VIX peak, 1w prior (vix selloff)
                     ('^VIX', '2025-04-01'),]
event_list_hivix_spy = [('SPY', '2018-01-28'), # Align VIX peak 1w prior (SPY)
                        ('SPY', '2025-04-01'),]
event_list_hivix3 = [('SPY', '2018-01-24'), # Align first VIX selloff
                     ('SPY', '2025-02-15'),]
event_list_test = [('RSP', '2025-05-27'), ('IWM', '2025-05-09')]

# TODO: Add election date
# TODO: Fix composite portfolios
# TODO: Add sharpe with rebalancing dates (returns on inverse-vol returns with rebalancing)
# TODO: Add a level toggle that doesn't subtract the first value
# TODO: Convert x-axis to date of first event
# TODO: Ensure first (latest) event is on top

event_list = event_list_hivix3
before = 21*12
after = 21*18
factor_name, event_date = event_list[0]

fig = draw_event_study(ret, event_list, before=before, after=after)
# fig.update_yaxes(autorange='reversed')
fig.show()
# fig.reset_index() #.to_clipboard()

In [ ]:
def flatten_multiindex(index, sep='_'):
    return index.map(lambda x: sep.join(map(str, x)))

def test_event_study(ret, event_list, before, after):
    # TODO: Run cumulative return inside the event study
    event_study = run_event_study(ret, event_list, before=before, after=after)
    # event_study.reset_index().pivot(index='day_offset', columns=('factor_name', 'event_date'), values='returns')
    df_cum = event_study.reset_index().pivot(index='day_offset', columns=['factor_name', 'event_date'], values='returns').cumsum()
    df_cum.columns = flatten_multiindex(df_cum.columns, sep=';')
    # event_study -= event_study.loc[0]
    df_cum -= df_cum.loc[0]
    fig = px.line(df_cum/100, template='plotly_white', title='Event Study')
    return px_format(fig)
    # return df_cum

ttt = test_event_study(ret, event_list=event_list_hispy, before=before, after=after)
ttt

In [ ]:
factor_data.cret.sel(factor_name='^VIX').to_pandas().quantile([0, 0.15, 0.5, 0.85, 1])


# Where's the TRADEWAR bottleneck?

In [ ]:
factor_data.sel(factor_name='TRADEWAR').ret.dropna('date')

factor_data.sel(factor_name=['FXF', 'FXY', 'GLD', 'XLP', 'XLY', '^VIX3M']).ret.to_pandas().dropna(how='all')